In [38]:
import sys
import os

import requests
import json

import pandas as pd

from web3 import Web3

# SCRIPT_DIR = os.path.dirname(os.path.abspath('__file__'))
PROJECT_DIR = os.path.dirname(os.path.abspath(''))
sys.path.append(os.path.dirname(PROJECT_DIR))

from pyDpi.util import setup
from pyDpi.util.libs import invoke_contract

In [39]:
def extratc_metada(item_metada):
    authors = []
    title = ''
    url = []
    ext_pid = []
    search_keys = []

    for metada in item_metada:
        if metada['key'] == 'dc.contributor.author':
            authors.append(metada['value'].upper().strip())
        if metada['key'] == 'dc.title':
            title = metada['value'].upper()
        if metada['key'] == 'dc.identifier.uri':
            url.append(metada['value'])
        if metada['key'] == 'dc.identifier.other':
            ext_pid.append(metada['value'].strip())
        if metada['key'] == 'dc.subject':
            search_keys.append(metada['value'].upper().strip())
    
    return title,authors,url,ext_pid,search_keys

curl -s -H "Accept: application/xml"  https://dspace.bc.uepb.edu.br/rest/collections

https://dspace.bc.uepb.edu.br/rest

https://wiki.lyrasis.org/display/DSDOC6x/REST+API#RESTAPI-RESTEndpoints

In [32]:
# url = 'https://dspace.bc.uepb.edu.br/rest/collections'
#payload = open("request.json")
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# r = requests.post(url, data=payload, headers=headers)
url_base = 'https://dspace.bc.uepb.edu.br/rest'
cmd_url = '/collections'

url = url_base + cmd_url
print(url)
headers = {'Accept': 'application/json'}
r = requests.get(url, headers=headers)
collections = json.loads(r.text)

https://dspace.bc.uepb.edu.br/rest/collections


seleciona a menor colecao

In [33]:
num_itens = 99999999
selected_collection_id = -1
for c in collections:
    if  c['numberItems'] < num_itens:
        if c['numberItems'] > 10:
            # print(c)
            selected_collection_id = c['id']
            num_itens = c['numberItems']


In [34]:
selected_collection_id,num_itens

(199, 11)

In [35]:
cmd_url = '/collections/'+str(selected_collection_id)+'/items'

url = url_base + cmd_url
print(url)
r = requests.get(url, headers=headers)
itens = json.loads(r.text)

len(itens)


https://dspace.bc.uepb.edu.br/rest/collections/199/items


11

In [45]:
w3 = setup.load_blockchain_driver()
deployed_contracts = setup.load_deployed_smart_contracts(w3)
dpid_db = deployed_contracts['PidDB.sol']
epid_db = deployed_contracts['ExternalPidDB.sol']

dpid_service = deployed_contracts['PIDService.sol']
epid_service = deployed_contracts['ExternalPIDService.sol']
sets_service = deployed_contracts['SearchTermService.sol']

chain_id,min_gas_price,pk = setup.get_exec_parameters()
account = w3.eth.account.privateKeyToAccount(pk)

/mnt/d/Dados/OneDrive/IBICT/workspace/D-pi/config.ini


In [37]:
def summary(author_db,item_db):
    saida = []
    for a in author_db.keys():
        ida = Web3.toHex(author_db[a][:16])
        # print(len(b))
        # ida = eid
        formated_id = ida[2:10]+'-'+ida[10:14]+'-'+ida[14:18]+'-'+ida[18:22]+'-'+ida[22:]
        linha = ['person', formated_id , ida ,a]
        saida.append(linha)
    
    for b in item_db.keys():
        idb = Web3.toHex(b[:16])
        # idb = eid
        # print(len(b))
        formated_id = idb[2:10]+'-'+idb[10:14]+'-'+idb[14:18]+'-'+idb[18:22]+'-'+idb[22:]
        linha = ['article',formated_id, idb ,item_db[b]]
        saida.append(linha)
    
    df = pd.DataFrame(saida)
    df.columns = [ 'types' , 'dpi_id' , 'id_16bytes' , 'desc']
    return df

In [40]:
author_db = {}
item_db = {}

for item in itens:
    item_id = item['id']
    cmd_url = '/items/'+str(item_id)+'/metadata'
    url = url_base + cmd_url
    # print(url)
    r = requests.get(url, headers=headers)
    item_metada = json.loads(r.text)
    title,authors,urls,ext_pid,search_keys = extratc_metada(item_metada)

    #pid
    recipt_tx = invoke_contract(w3,account,chain_id, dpid_service , 'assingUUID')
    pid_id = recipt_tx['logs'][0]['topics'][1]

    for key in search_keys:
        recipt_tx = invoke_contract(w3,account,chain_id, sets_service , 'get_or_create_search_term' , key )
        # sk_id = recipt_tx['logs'][0]['topics'][1]
        recipt_tx = invoke_contract(w3,account,chain_id, dpid_service , 'addSearchTerm' , pid_id[:16], key )
    
    for epid in ext_pid:
        recipt_tx = invoke_contract(w3,account,chain_id, dpid_service , 'addExternalPid' ,pid_id[:16],'CDD',epid)
    
    for url in urls:
        recipt_tx = invoke_contract(w3,account,chain_id, dpid_service , 'add_externalLinks', pid_id[:16], url)

    recipt_tx = invoke_contract(w3,account,chain_id, dpid_service , 'set_payload', pid_id[:16], title)

    item_db[pid_id] = url

    for author in authors:
        try:
            author_db[author]
        except KeyError:
            recipt_tx = invoke_contract(w3,account,chain_id, dpid_service , 'assingUUID')
            author_id = recipt_tx['logs'][0]['topics'][1]
            recipt_tx = invoke_contract(w3,account,chain_id, dpid_service , 'set_payload', author_id[:16], author)
            author_db[author] = author_id

In [49]:
dpi_obect = dpid_db.caller.get('0x650e3cc9787035c822ad6477980c8adb')

for i in dpi_obect[2]:
    print(epid_db.caller.get(i))

(b'7\xdf\x1a\xd6-\xbf\x88\x12\x85\x1e<J\x15\xf1\xa4\xd0\xa7\xec\x03sA6\x92\x84\x86u\xad\xaecQ\x00\xa3', b'e\x0e<\xc9xp5\xc8"\xaddw\x98\x0c\x8a\xdb', 'CDD 363.73', 'CDD', '0xf17f52151EbEF6C7334FAD080c5704D77216b732')


In [42]:
summary(author_db,item_db)

,types,dpi_id,id_16bytes,desc
0,person,4539a771-bcd9-ed68-26f0-606bca156df5,0x4539a771bcd9ed6826f0606bca156df5,"SOUZA, JOÃO BATISTA DE"
1,person,fe5d42e3-cbd4-7a6a-2834-26f5672de4e5,0xfe5d42e3cbd47a6a283426f5672de4e5,"LIMA, VICTOR HUGO MOREIRA DE"
2,person,57ec1e35-36d8-39e0-5f7a-730049153bd9,0x57ec1e3536d839e05f7a730049153bd9,"CONFESSOR, MAINE VIRGÍNIA ALVES"
3,person,7baeb22a-ab1d-6e38-f2a5-7813cfd93b36,0x7baeb22aab1d6e38f2a57813cfd93b36,"MASCARANHAS, ROSIMARY RAMOS DE OLIVEIRA"
4,person,d53cb1c5-d20d-efa5-aa97-42d105fd3689,0xd53cb1c5d20defa5aa9742d105fd3689,"ALVES, RAFAEL MACHADO DE ARAÚJO"
5,person,6d966513-a60b-2250-ed54-0d5df28e8950,0x6d966513a60b2250ed540d5df28e8950,"OGATA, IGOR SOUZA"
6,person,0b3ecfc0-9fbd-2277-fd74-df8101590467,0x0b3ecfc09fbd2277fd74df8101590467,"ANSELMO, MARIA DA GLÓRIA VIEIRA"
7,person,736fa331-09f5-d159-74da-2432c417cc92,0x736fa33109f5d15974da2432c417cc92,"NERY, JANIELE FRANÇA"
8,person,f4686f06-8c77-d725-330b-008840ede41d,0xf4686f068c77d725330b008840ede41d,"PÊ, PATRÍCIA RODRIGUES"
9,person,3e96a1fd-5de2-f855-69a9-1f95cb68c4f3,0x3e96a1fd5de2f85569a91f95cb68c4f3,"FREIRE, VITÓRIA DE ANDRADE"


## w3